# SPAM in Gmail is an example of Text classifier. Dataset taken from http://www.cs.cornell.edu/people/pabo/movie-review-data/

import numpy as np
import re
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

In [87]:
# Importing dataset
reviews= load_files('txt_sentoken/')
x,y= reviews.data,reviews.target

# persisting dataset by storing as pickle file, save time in loading huge amount of data

In [88]:
with open('x.pickle','wb') as f:             #wb write byte
    pickle.dump(x,f)

In [89]:
with open('y.pickle','wb') as f:             
    pickle.dump(y,f)

# Unpickling the dataset

In [90]:
with open('x.pickle','rb') as f:             #rb read byte
    x=pickle.load(f)

In [91]:
with open('y.pickle','rb') as f:             #rb read byte
    y=pickle.load(f)

# Creating the corpus by preprocessing the data

In [92]:
corpus= []
for i in range(0,len(x)):
               review=re.sub(r"\W"," ", str(x[i]))
               review=review.lower()
               review= re.sub(r'\s+[a-z]\s+',' ',review)     # remove all single characters
               review= re.sub(r'^[a-z]\s+',' ',review)
               review= re.sub(r'\s+',' ',review)
               corpus.append(review)
               
    

# Transform data into BOW Model

In [93]:
from sklearn.feature_extraction.text import CountVectorizer

In [94]:
vectorizer= CountVectorizer(max_features=2000, min_df=3, max_df=0.6,stop_words=stopwords.words('english'))

In [95]:
x= vectorizer.fit_transform(corpus).toarray()

In [96]:
print(x)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [2 0 1 ..., 0 0 0]]


# Transform BOW model into TF-IDF Model

In [97]:
from sklearn.feature_extraction.text import TfidfTransformer

In [98]:
transformer= TfidfTransformer()

In [99]:
x=transformer.fit_transform(x).toarray()

In [100]:
print(x)

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.06887219  0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.12007883  0.          0.06321361 ...,  0.          0.          0.        ]]


# Creating Training and Testing Dataset

In [101]:
from sklearn.model_selection import train_test_split

In [102]:
text_train, text_test, sent_train, sent_test = train_test_split(x,y,test_size=0.2,random_state=0)        # size=0.2 means 80%data is for training, train_test+split returns 4 arguments

# Training Classifier

In [103]:
from sklearn.linear_model import LogisticRegression

In [104]:
classifier= LogisticRegression()

In [105]:
classifier.fit(text_train, sent_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# Testing Model Performance

In [106]:
sent_predictions= classifier.predict(text_test)

In [107]:
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(sent_test,sent_predictions)

In [108]:
(cm[0][0]+cm[1][1])/4

84.75

# Saving Model by pickling the classifier

In [109]:
with open('classifier.pickle','wb') as f:
    pickle.dump(classifier,f)

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer(max_features=2000, min_df=3, max_df=0.6,stop_words=stopwords.words('english'))
x= vectorizer.fit_transform(corpus).toarray()

In [111]:
with open('tfidfmodel.pickle','wb') as f:
    pickle.dump(vectorizer,f)

# Importing and using this model

In [115]:
#Unoickling the classifier and vectoriser
with open('classifier.pickle','rb') as f:
    clf= pickle.load(f)

In [116]:
with open('vectorizer.pickle','rb') as f:
    tfidf= pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'vectorizer.pickle'

In [117]:
sample= ["you are a nice person man,have a good life"]

In [118]:
sample= tfidf.transform(sample).toarray()

NameError: name 'tfidf' is not defined

In [ ]:
print (clf.predict(sample))